# Operaciones básicas con DataFrames

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

Leemos el fichero CSV utilizando el delimitador por defecto de Spark (","). La primera línea contiene encabezados (nombres de columnas) por lo que no es parte de los datos y debemos indicarlo con la opción header.

In [ ]:
# Esto no hace nada: la lectura es lazy así que no se lee en realidad hasta que ejecutemos una acción sobre flightsDF
# Solamente se comprueba que exista el fichero en esa ruta, y se leen los nombres de columnas
flightsDF = spark.read.option("header", "true")\
                 .csv("gs://ucm__bucket/data/flights-jan-apr-2018.csv")

In [ ]:
# Veamos el esquema (nombre y tipo de dato de cada columna). Esto son solamente metadatos, por lo que no es ninguna acción.
flightsDF.printSchema()

Todas las columnas son cadenas de caracteres porque no hemos indicado el tipo de dato para cada columna ni tampoco le hemos pedido a Spark que intente inferirlo a partir de los datos. No queremos que todas sean string porque hay algunas numéricas que deberían ser tratadas como tales. Vamos a intentar inferir el esquema. Esto supone una lectura un poco más lenta y también es más lento que una tercera opción que consiste en indicar explícitamente el esquema para los datos en el momento de la lectura, que es la opción recomendada si sabemos de antemano qué tipo va a tener cada columna. Si lo hiciésemos de esa manera, en caso de que no se pueda leer con ese esquema obtendríamos un error.

In [2]:
from pyspark.sql import functions as F

flightsDF = spark.read\
                 .option("header", "true")\
                 .option("inferSchema", "true")\
                 .csv("gs://ucm__bucket/data/flights-jan-apr-2018.csv") # pon aquí la ruta en tu bucket

# Ensuciamos a propósito la variable ArrDelay para que pase a ser un string como suele pasar con frecuencia
flightsDF = flightsDF.withColumn("ArrDelay",\
                                 F.when(F.rand(123) < 0.1, "NA").otherwise(F.col("ArrDelay")))

flightsDF.printSchema()

root
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCity: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DestCity: string (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)



Ahora tiene mejor pinta, aunque todavía hay algunas columnas cuyo tipo de dato sigue siendo string cuando la intuición nos dice que deberían ser enteros.

## Operaciones básicas con Data Frames

No todas las operaciones sobre un DataFrame de Spark requieren llevar a cabo cálculos en los executors. Algunas simplemente consultan *metadatos* del DataFrame. Puesto que los DataFrames siempre se manejan desde el driver, y toda la metainformación (como por ejemplo nombres y tipos de las columnas, número de particiones, etc) está en el driver, consultar dicha información no requiere calcular nada en los executors.

### Consultamos las columnas que tiene el DF

In [ ]:
flightsDF.columns

### Contamos el número de filas

Es una de las primeras cosas que nos preguntamos sobre un dataset: número de filas y columnas (cuántos ejemplos y cuántas variables tenemos para describirlos). Puesto que vamos a llevar a cabo varias transformaciones al DataFrame `flightsDF` a partir de este punto, vamos a usar `cache`() para que Spark lo mantenga en memoria en lugar de liberar la memoria ocupada tras cada acción.

In [ ]:
# Extraemos los nombres de columna. Esto son solo metadatos de DataFrame, y están en el driver. No es necesaria ninguna
# operación sobre el cluster para recuperar la variable interna columns de cualquier DataFrame
print("Los datos tienen {0} columnas".format(len(flightsDF.columns)))

flightsDF.cache()        # Esta línea no hace cálculos, pero Spark anota que debe mantener este DF en memoria tras la primera vez que sea materializado
rows = flightsDF.count() # Esto es una acción que obligará a que flightsDF sea materializado. Para ello, habrá que llevar a cabo
                         # las transformaciones que lo generan en la celda anterior: read y withColumn, que están pendientes

print("Los datos tienen {0} filas".format(rows))

In [ ]:
flightsDF.is_cached       # consultamos el flag booleano que hay en el driver

In [ ]:
flightsDF.count()

¿Qué pasaría si no cacheamos flightsDF? Probemos a ejecutar las celdas anteriores sin cachear y cacheando.

### Seleccionar columnas por nombre

In [ ]:
resultDF = flightsDF.select("Month", "DayofMonth", "ArrTime")
resultDF.show(10) # los nombres son sensibles a mayúsculas

### Filtramos (retenemos) filas en base a los valores de una o varias columnas

Esto se puede hacer con sintaxis de SQL puro, o bien con operaciones sobre objetos columna

La mayoría de las transformaciones están definidas en el paquete `pyspark.sql.functions`, por lo que es frecuente importar el paquete completo con un alias, como `F`, en lugar de importar cada función individual. A partir de ese momento usamos `F.` antes del nombre de cada función, para decirle a python dónde buscar esa función.

Mostramos también cómo encadenar varias transformaciones, sean la misma o distintas transformaciones combinadas una tras otra.

In [ ]:
flightsJanuary20 = None
flightsJanuary20 = flightsDF\
                      .where("DayofMonth = 20 and Month = 1")\
                      .withColumn("DistKm", 1.6 * flightsDF["Distance"])\
                      .withColumn("DobleDistancia", 2 * F.col("DistKm"))
#                      .select("Month", "ArrTime", "DistKm") # encadenamos dos transformaciones: esto no desencadena ninguna operación

In [ ]:
flightsJanuary20.printSchema()

In [ ]:
# La función col se utiliza para decir que nos estamos refiriendo a la columna cuyo nombre se pasa como argumento
flightsJanuary20 = flightsDF\
                      .where((F.col("DayofMonth") == 20) & (F.col("Month") == 1))\
                      .select("Month", "ArrTime")\
                      .cache() # encadenamos dos transformaciones: esto no desencadena ninguna operación

# Cúantos vuelos hay el 20 de enero de 2018
# La operación count() es una acción, así que obliga a materializar flightsJanuary20. Para ello es necesario ejecutar
# las transformaciones where() y select() que pusimos en esta celda, aplicadas a flightsDF. De hecho, si no hubiésemos
# cacheado flightsDF en las celdas anteriores, también habría que materializarlo otra vez, y para eso se leería de 
# nuevo el CSV desde HDFS
rowsJanuary20 = flightsJanuary20.count()

print("Hubo {0} vuelos el 20 de enero de 2018".format(rowsJanuary20))

# Esto es otra acción aplicada sobre el DataFrame flightsJanuary. Como flightsJanuary NO ha sido cacheado,
# entonces las operaciones "where" y "select" se necesitan ejecutar DE NUEVO para poder hacer el show()
flightsJanuary20.show(3)

**No olvidemos cachear el DataFrame cuando tengamos previsto hacer varias operaciones sobre él, o de lo contrario estaremos repitiendo muchas veces los cálculos previos que llevaron a ese DataFrame!!**

In [ ]:
# También podemos indicar el filtrado como un string con un trozo de código SQL
# Recordemos que where y filter son exactamente equivalentes
flightsJanuary31 = flightsDF.filter("DayofMonth = 31 and Month = 1") # transformación filter: Spark no ejecuta nada

flightsJanuary31.count() # acción count: obliga a materializar flightsJanuary31 para lo cual se tiene que ejecutar filter

<div class="alert alert-block alert-info">
<b>RECUERDA:</b> Spark hace todas estas operaciones de manera distribuida en el cluster, por tanto cada nodo del cluster está filtrando filas de entre aquellas que están presentes en ese nodo (más precisamente, en ese executor). Cada executor envía al driver el recuento de cuántas filas ha filtrado, y los resultados son agregados en el driver para mostrar el recuento total.
</div>

<div class="alert alert-block alert-success">
<b>TU TURNO</b>: muestra por pantalla el retraso en la llegada, el aeropuerto de origen y de destino de aquellos vuelos que tuvieron lugar en Domingo y con un retraso a la llegada mayor de 15 minutos. Muestra el esquema de dicho DataFrame resultante.

</div>

In [ ]:
flightsDomingoDF = flightsDF.filter("DayOfWeek = 7 and ArrDelay > 15")\
                            .select("ArrDelay", "Origin", "Dest")
flightsDomingoDF.show()

### No utilices `show` al final de una secuencia de transformaciones

<div class="alert alert-block alert-warning">
<b>CUIDADO</b>: si pretendes almacenar el DF resultante de una secuencia de transformciones encadenadas, no pongas nunca `show()` al final de esa secuencia! La acción `show()` devuelve `None` (valor nulo de Python) y por tanto, lo que estarás almacenando en esa variable será el nulo y no podrás seguir utilizando la variable. Lo correcto es almacenar el resultado de la última transformación, y luego imprimir el DF.
</div>

In [ ]:
flightsJanuary20 = flightsDF\
                      .where((F.col("DayofMonth") == 20) & (F.col("Month") == 1))\
                      .select("Month", "ArrTime")\
                      .show()  # ¡¡¡ ERROR !!!

flightsJanuary20.select("Month") ### ERROR: flightsJanuary20 almacena None

### Existen distintas sintaxis que podemos usar para referirnos a objetos Column de un DF de Spark

1. Podemos usar `F.col("nombreColumna")`
2. También `nombreDataFrame.nombreColumna`
3. Y también `nombreDataFrame["nombreColumna"]`

Pero las dos últimas tienen un problema, como podemos ver abajo: sólo pueden utilizarse sobre un DF que tenga nombre, es decir, que esté almacenado en alguna variable. Pero no todos lo están...

Por ejemplo: *si intentamos aplicar las opciones 2 y 3 sobre una columna que hemos creado justo en un **DataFrame intermedio**, no podremos*, porque cualquier DF intermedio en una secuencia de transformaciones *encadenadas* **no tiene nombre**, y por tanto es imposible referirnos a una columna como `nombreDF.nombreColumna` porque no tenemos ninguna manera de referirnos a ese DataFrame.

In [ ]:
# no puedo poner flightsDF.DistKm porque DistKm no pertenece a flightsDF !!

flightsDF.withColumn("DistKm", 1.6*F.col("Dist"))\
         .select(flightsDF.Origin, flightsDF.Dest, F.col("DistKm"))

En cambio, la sintaxis `F.col("nombreColumna")` **funciona siempre**, ya que está indicando un objeto columna que realmente *toma contexto* en el momento en el que lo utilizamos dentro de alguna transformación (por ejemplo select, withColumn, ...). Si el DF sobre el cual estamos aplicando la transformación, aunque dicho DF no tenga nombre, sí posee la columna a la que nos referimos dentro de F.col, entonces la operación es correcta y se puede realizar sin problema.

In [ ]:
columna_orig = F.col("origDistintos")

distinctOrigins = flightsDF.select(F.col("ArrDelay"), flightsDF.Distance, flightsDF["Origin"])\
                            .select(F.lit(F.min("ArrDelay")).alias("minimo"),
                                    F.mean("Distance").alias("distMedia"),
                                    F.countDistinct("Origin").alias("origDistintos"))\
                            .select("minimo", F.col("distMedia"), F.col("origDistintos"))\
                            .withColumn("mitadOrigin", columna_orig / 2)  # OJO: no se pueden mezclar columnas de una sola fila con columnas normales
distinctOrigins.show()


### Crear y seleccionar columnas al vuelo

Podemos usar `select` no solo para seleccionar columnas existentes sino también para crear nuevas columnas al vuelo y a la vez seleccionarlas. Vamos a seleccionar Origin y Dest, que ya existen, a la vez que seleccionamos una tercera columna que estamos creando al vuelo, con la transformación de la distancia a km. Le ponemos como nombre "DistanceKm" en ese momento que la estamos creando, mediante la función `alias`. Si no usamos `alias`, Spark le pondrá a la nueva columna un nombre por defecto que en este caso sería "1.6 * Distance". Se recomienda usar alias para dar nombre a las nuevas columnas.

In [ ]:
# Esto es una posibilidad: selectExpr poniendo expresiones SQL puras
flightsDF.selectExpr("Origin", "Dest", "1.6*Distance AS DistanceKm").show()

<div class="alert alert-block alert-success">
<b>TU TURNO</b>: crear un DataFrame con tres columnas seleccionando "Origin", "OriginCity" y una nueva columna de tipo string creada concatenando Origin y OriginCity con un guión "-". Utilizar `withColumn` y dentro la función `concat_ws` (concatenar con separador) con sintaxis <a target="_blank" href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.concat_ws">F.concat_ws("-", columna1, columna2)</a> del paquete pyspark.sql.functions.
</div>

In [ ]:
concatenadoDF = flightsDF.withColumn("concatenado", F.concat_ws("-", F.col("Origin"), F.col("OriginCity")))\
                         .select(F.col("Origin"), F.col("OriginCity"), F.col("concatenado"))
concatenadoDF.show()

### Cuidado: no es posible seleccionar columnas de "muchas filas" y de "una sola fila" a la vez

Cuando aplicamos una función de agregación a una columna completa (p.ej. el máximo de una columna), el resultado es otro objeto Column de *una sola fila*. No se puede seleccionar ese objeto a la vez que una columna que sí tiene múltiples filas.

In [ ]:
# Esto es correcto porque todas las columnas seleccionadas o creadas tienen una sola fila
minimosDF = flightsDF.select(F.max("ArrDelay").alias("maxDelay"),
                             F.min("ArrDelay").alias("minDelay"))

# Esto no es correcto porque la primera columna tiene muchas filas pero la columna maxDelay tiene una sola fila
badDF = flightsDF.select(F.col("ArrDelay"), # ERROR!!!
                         F.max("ArrDelay").alias("maxDelay"))

### Seleccionar filas únicas

Para hacerse mejor idea de cómo es una variable categórica, es lógico querer cuántos valores distintos existen en nuestro dataset. Si consideramos todas las columnas, sería raro tener dos filas exactamente iguales en todos los valores, pero si seleccionamos solamente una o unas pocas columnas, podemos ver cuántas combinaciones distintas de valores de esas columnas se dan en el dataset.


In [ ]:
distinctFlights = flightsDF.distinct()  # distinct es una transformación que devuelve el DF sin las filas repetidas
distinctFlightsCount = distinctFlights.count() # count es una acción y provoca que se ejecute la transformación distinct

print("Hay {0} filas distintas".format(distinctFlightsCount))

Si seleccionamos solo las columnas `Origin` y `Dest` y nos quedamos con las filas distintas (quitamos repetidos), entonces obtenemos un DataFrame con los posibles aeropuertos de origen y destino, es decir, aquellos trayectos que existen en un vuelo (pueden aparecer varias veces porque seguramente existirán muchos vuelos a lo largo de 2018 entre un origen y un destino)

<div class="alert alert-block alert-success">
    <b>TU TURNO</b>: ¿<b>cuántas</b> combinaciones de Origin y Dest existen?
</div>

**Respuesta en la siguiente celda:**

In [ ]:
countOriginDests = flightsDF.select("Origin", "Dest").distinct().count()

print(f"Hay {countOriginDests} combinaciones de un aeropuerto de origen y uno de destino")

Aunque tengamos 2.5 millones de filas, solo hay 5795 combinaciones distintas de un aeropuerto de origen y otro de destino.

<div class="alert alert-block alert-success">
    <b>TU TURNO</b>: ¿<b>cuántos</b> aeropuertos de origen existen?
</div>

In [ ]:
distinctOrigins = flightsDF.select("Origin")\
                           .distinct()\
                           .count()
        
flightsDF.select(F.countDistinct("Origin").alias("origDistintos")).show()

print(f"Hay {distinctOrigins} aeropuertos desde los que puede partir un vuelo")

<div class="alert alert-block alert-success">
<b>TU TURNO</b>: vamos a hacer esto para cada columna, en un bucle, para hacernos a la idea de cuántos valores hay. Esto solo tiene sentido en realidad para columnas categóricas y no para numéricas donde casi todos los valores serán distintos.
</div>

In [ ]:
for columnName in flightsDF.columns:

    distinctValues = flightsDF.select(columnName).distinct().count()
    
    # No olvidéis indentar este comando para indicar que está dentro del cuerpo del bucle
    print(f"Existen {distinctValues} valores distintos en la columna {columnName}")


### Otra opción: función que recibe un DF y devuelve otro DF que tiene una sola fila con el conteo de valores distintos de cada columna

Utilizamos list-comprehension para crear una **lista de objetos Column, que todavía no están ligados a ningún DataFrame** hasta que no utilicemos esa lista en alguna transformación (por ejemplo select) y entonces esos objetos columna **toman contexto**.

In [ ]:
def df_conteos(sparkDF):
    # creamos una lista de objetos columna sin contexto (no pertenecen a ningún DF)
    colsRecuento = [F.countDistinct(c).alias(c) for c in sparkDF.columns]
    # Y ahora lo utilizamos en select y entonces Spark entiende a qúe DF nos referimos
    return sparkDF.select(colsRecuento)

conteosDF = df_conteos(flightsDF)
conteosDF.show()

### ¿Y si quisiéramos mostrar el número de valores solamente de aquellas columnas que sean categóricas?

La variable interna `dtypes` que tienen todos los DF nos da una lista de parejas de ('nombreColumna', 'tipoDato') con la que podemos filtrar las columnas de tipo string para aplicar la operación de la celda anterior solamente a esas columnas. Hacemos list comprehension y un poco de pattern-matching para extraer directamente los dos campos de cada pareja en el bucle, en lugar de extraer un solo objeto tupla.

In [ ]:
def df_conteos_categoricas(sparkDF):
    tipos = sparkDF.dtypes
    colsRecuento = [F.countDistinct(c).alias(c) for (c, tipo) in sparkDF.dtypes if tipo == "string"]
    return sparkDF.select(colsRecuento)

conteosCategoricasDF = df_conteos_categoricas(flightsDF)
conteosCategoricasDF.show()

### Crear una nueva columna o reemplazar una existente por el resultado de operar con columnas existentes

Tenemos las distancias en millas. Vamos a convertirlas a km multiplicando las millas por 1.61. De nuevo, cada nodo del cluster hará esta operación localmente con los datos que se encuentran en ese nodo. El DataFrame resultante estará repartido en los mismos nodos. **No hay movimiento de datos ya que no se necesita ninguna información que no esté presente en ese nodo** para efectuar la operación.

In [ ]:
# withColumn is a transformation returing a new DataFrame with one extra column appended on the right
flightsWithKm = flightsDF.withColumn("DistanceKm", F.col("Distance") * 1.61)

flightsWithKm.printSchema()

flightsWithKm.show(3)

El día de la semana es una variable entera. Utilizar un entero o una variable categórica es una decisión que depende de qué tipo de modelo vayamos a ajustar. ¿Tiene sentido considerar días de la semana "más grandes" o "más pequeños", es decir, algo que se incrementa conforme "se incrementa" el día de la semana de 1 a 7? Aquí vamos a **reemplazar** la columna `DayOfWeek` que ya existía, por una versión categórica como strings. Utilizamos `withColumn` pasándole como primer argumento el nombre de una columna que ya existe, lo cual indica que queremos reemplazarla, en el mismo lugar que ocupaba.

In [ ]:
flightsCategoricalDay = flightsDF.withColumn("DayOfWeek", F.when(F.col("DayOfWeek") == 1, "Monday")\
                                                           .when(F.col("DayOfWeek") == 2, "Tuesday")\
                                                           .when(F.col("DayOfWeek") == 3, "Wednesday")\
                                                           .when(F.col("DayOfWeek") == 4, "Thursday")\
                                                           .when(F.col("DayOfWeek") == 5, "Friday")\
                                                           .when(F.col("DayOfWeek") == 6, "Saturday")\
                                                           .otherwise("Sunday"))

flightsCategoricalDay.printSchema() # the column is still in the same position but has now string type

flightsCategoricalDay.select("DayOfWeek", "DepTime", "ArrTime").show()

<div class="alert alert-block alert-info">
    <b>CONSEJO:</b> El proceso de crear nuevas variables o <i>features</i> a partir de otras existentes, incluso incorporar variables de fuentes de datos externas (datos públicos) y de limpiar o reemplazar variables tras normalizarla se denomina genéricamente <i>feature engineering</i> (ingeniería de variables). A veces tiene mucho que ver con conocimientos específicos del dominio, aunque también con trucos estadísticos para normalizar siguiend métodos bien conocidos.
</div>

La función `when` es muy común para re-categorizar una variable o para crear nuevas variables categóricas a partir de condiciones complejas acerca de lo que les ocurre a los valores de otras columnas en esa misma fila.

<div class="alert alert-block alert-success">
<b>TU TURNO</b>: crea una variable categórica de tipo string con dos categorías indicando si el día de la semana es laborable o es fin de semana. Los valores deben ser "laborable" o "finde". Utiliza `withColumn` y `when`. Será laborable cuando DayOfWeek esté entre 1 y 5, y fin de semana cuando sea 6 o 7 (en resumen: "en otro caso" es fin de semana). 
</div>

In [ ]:
flightsFindeLaborable = flightsDF.withColumn("Laborable", F.when(F.col("DayOfWeek")<= 5, "Laborable")\
                                                           .otherwise("Finde"))

flightsFindeLaborable.printSchema()
flightsFindeLaborable.select("DayOfWeek", "Laborable").show()

### Otra posibilidad: utilizar .isin() aplicado al objeto columna para expresar la condición del when

In [ ]:
flightsCategoricalDay = flightsCategoricalDay.withColumn("TypeOfDay", 
                            F.when(~(F.col("DayOfWeek").isin({'Saturday','Sunday'})), "laborable")\
                             .otherwise("finde"))
flightsCategoricalDay.show(3) 

In [ ]:
findeDF = flightsDF.withColumn("finde", F.when(F.col("DayOfWeek") <= 5, "laborable")\
                                         .otherwise("finde"))

### Para reemplazamientos usando un solo diccionario, que tenga {'categoríaOriginal': 'nuevoValor'}. Ese diccionario puede haberse leído de algún fichero o similar

In [ ]:
from pyspark.sql import functions as F
from itertools import chain

equivalencias = {
    'LAX': 'LosAngeles', 
    'SFO': 'SanFrancisco'
}

mapping_expr = F.create_map([F.lit(x) for x in chain(*equivalencias.items())]) # creamos una columna de tipo diccionario

# Ahora utilizamos esa columna de diccionario, pidiéndole los elementos asociados a las claves que indica la columna Origin 
# (que son las categorías originales). Por tanto nos devolverá otra nueva columna con todos los valores asociados a dichas claves, fila a fila
flightsDF.withColumn("OriginRecat", mapping_expr.getItem(F.col("Origin")))\
         .select("Origin", "OriginRecat").distinct().show()

### Se puede usar la sintaxis de operador punto y similares para referirse a un objeto columna, solo cuando el DF al que pertenece esa columna tiene nombre

OJO: en el siguiente ejemplo, no puedo poner flightsDF.DistKm porque DistKm no pertenece a flightsDF sino que pertenece al DF devuelto por `withColumn(...)` y dicho DF no tiene ningún nombre

In [ ]:
# no puedo poner flightsDF.DistKm porque DistKm no pertenece a flightsDF !!

flightsDF.withColumn("DistKm", 1.6*F.col("Dist"))\
         .select(flightsDF.Origin, flightsDF.Dest, F.col("DistKm"))

### Convertir el tipo de dato de una columna

Con frecuencia, Spark no infiere correctamente el tipo de cada columna. 
* A veces ocurre que una columna numérica no se reconozca adecuadamente porque los datos tienen "NA" (un string) que quería significar data faltante en el dataset original. "NA" no es un string especial para Spark, por lo que simplemente reconoce que la columna tiene tanto enteros como strings, y el tipo más general que infiere es string para esa columna. 
* Pero no es el caso aquí porque no se da esto, y por eso hemos ensuciado al principio los datos a propósito. Vamos a limpiarlos y a convertir la columna `ArrDelay` a DoubleType como debe ser.

In [ ]:
naCount = flightsDF.where("ArrDelay = 'NA'").count()

# Esto es sintaxis SQL, pero también podríamos haberla llamado como .where(F.col("ArrDelay") == "NA"). Ambas son equivalentes.

print("Hay ", naCount, "filas con NA en ArrDelay")

In [ ]:
#from pyspark.sql.types import DoubleType
from pyspark.sql import types as T

flightsDF = flightsDF.where((F.col("ArrDelay") != "NA"))\
                     .withColumn("ArrDelay", F.col("ArrDelay").cast(T.DoubleType()))

### Ordenación respecto a una columna

Es una transformación que nos devuelve otro DataFrame ordenado según la(s) columna(s) indicada(s), sea de forma ascendente o descendente. El DataFrame original no se modifica (al igual que ocurre en cualquier transformación). Se puede ordenar en base a una columna numérica (lo más frecuente) o también categórica (los strings se ordenan alfabéticamente). Para ordenar sí es necesario enviar información de unos nodos a otros puesto que no sabemos si existen o no datos mayores o menos que el valor que tenemos en el nodo (o más precisamente, en el worker).

In [ ]:
# Ordenamos los vuelos según ArrDelay
sortedDF = flightsDF.orderBy("ArrDelay")  # equivalente: flightsDF.orderBy(F.col("ArrDelay"))

# Orden ascendentemente por aeropuerto de origen ("Origin") y deshago los empates por arr_delay descendentemente
sortedDF = flightsDF.orderBy(F.col("Origin"), F.col("ArrDelay").desc())  # equivalente: flightsDF.orderBy(F.col("ArrDelay"))

sortedDescDF = flightsDF.orderBy("ArrDelay", ascending = False)
sortedDescDF.select("ArrDelay", "Origin", "Dest").show(10)

### Traemos una sola fila al driver como objeto Row y accedemos a los campos con el operador . 

In [ ]:
primeraFila = sortedDescDF.first()

In [ ]:
primeraFila.ArrTime

In [ ]:
primeraFila.asDict()

In [ ]:
listaRows = sortedDescDF.take(20)  # devuelve una lista de Python, de objetos Row
distancias = [r.Distance for r in listaRows]  # usamos sintaxis de listas por comprensión
distancias

In [ ]:
listaLAX = flightsDF.where("Origin = 'LAX'").collect() # ¡¡¡¡ CUIDADO !!!!
len(listaLAX)

In [ ]:
listaLAX[0:4]

### Vamos a comprobar que al llevar al driver las 10 primeras filas, el DF estaba ordenado así que esas filas están ordenadas

In [ ]:
lista = ordenadoDF.take(10) # esto devuelve una lista de python

### Funciones de agregación sobre el DataFrame completo

Spark tiene implementaciones distribuidas y paralelas de funciones de agregación frecuentes como la media, min, max y desviación típica entre otras. Todas estas funciones reciben como argumento el objeto columna sobre el que la función debe aplicarse. Lo más habitual es aplicarlas para agregar por grupos, pero esto lo veremos en el segundo notebook.

Vamos a seleccionar vuelos con `ArrDelay` mayor de 15 minutos, y con ellos vamos a crear columnas con el min, max, media y desviación típica de la columna `ArrDelay`. Crearemos y seleccionaremos dichas columnas al vuelo, como vimos en la sección anterior.

In [ ]:
flightsDF.printSchema()

In [ ]:
flightsDF.count()

In [ ]:
# First we select those flights with at least 16 minutes of delay and then compute the aggregations
agregacionesDF = flightsDF.where(F.col("ArrDelay") > 15)\
                          .selectariance("ArrDelay").alias("varArrDelay")
                            )

agregacionesDF.show()

Existe una función de Spark que hace casi todo esto por nosotros, llamada `summary`. Lo hace para cada columna numérica que encuentre en el dataset. Es también una transformación que devuelve un nuevo DataFrame con las métricas de resumen, sin modificar el DataFrame original.

In [ ]:
summariesDF = flightsDF.summary().cache()
summariesDF.show()

In [ ]:
summariesDF.printSchema()

In [ ]:
print(summariesDF.count()) # vamos a contar el número de filas del DF resumen

### Conversión a Pandas

En general es complicado leer los resultados que muestra Spark. En ocasiones interesa convertirlos a un dataframe de Pandas, aunque esto implica traer todas las filas al driver, lo cual **debe hacerse con mucho cuidado y solo en casos en los que estemos seguros de que el DataFrame de Spark es pequeño y no desbordará la memoria del driver**.

<div class="alert alert-block alert-info">
<p><b>CONSEJO</b>: el concepto de dataframe como tabla con filas y columnas existe en muchos lenguajes de programación (pandas de Python, dataframes de R, DataFrames de Spark). Sin embargo, Spark maneja DataFrames que están físicamente distribuidos en las memorias RAM de las máquinas del cluster, lo cual no tiene nada que ver con lo que hace la biblioteca Pandas o R que se ejecutan en una sola máquina. Convertir un DataFrame de Spark en un dataframe de Pandas implica llevar todas las filas al driver, lo cual podría resultar en una excepción Out-of-Memory si el contenido del DataFrame es más grande que la memoria RAM de la máquina en la que se está ejecutando el programa driver. En este caso y en la mayoría de casos, se suele utilizar para mostrar resúmenes o agregados ya calculados previamente, y que sabemos que ocupan poco, con lo que no existe riesgo.</p>

<p>Esta operación también es muy frecuente cuando queremos representar gráficamente el contenido de un DataFrame de Spark. No existen funciones gráficas en Spark, por lo que tenemos que convertirlo en un dataframe de Pandas y utilizar las funciones gráficas de Python habituales (matplotlib, Seaborn o incluso la propia biblioteca Pandas) para mostrar lo que necesitemos.
</p>
</div>

In [ ]:
summariesDF.is_cached

In [ ]:
flightsPd = summariesDF.toPandas()

In [ ]:
flightsPd

In [ ]:
otroDF = spark.createDataFrame(flightsPd)